In [1]:
import numpy as np
import pandas as pd
import math

# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.max_rows = None  # 모든 row 내용 print
pd.options.display.max_rows = 20

### Load the dataset with pandas
The car data set we will be using in this lab is provided as a comma separated file without a header row.  In order for each column to have a meaningful header name we must provide it.  We get the information about the columns from the [Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile).

We will use the features of the car, to try to predict its price.

In [3]:
# Provide the names for the columns since the CSV file with the data does not have a header row.
feature_names = ['symboling', 'normalized-losses', 'make', 'fuel-type',
        'aspiration', 'num-doors', 'body-style', 'drive-wheels',
        'engine-location', 'wheel-base', 'length', 'width', 'height', 'weight',
        'engine-type', 'num-cylinders', 'engine-size', 'fuel-system', 'bore',
        'stroke', 'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
        'highway-mpg', 'price']

In [4]:
# Load in the data from a CSV file that is comma separated.
car_data = pd.read_csv('https://storage.googleapis.com/mledu-datasets/cars_data.csv',
                     sep=',', names=feature_names, header=None, encoding='latin-1')

 We'll then randomize the data, just to be sure not to get any pathological ordering effects that might harm the performance of Stochastic Gradient Descent.

In [5]:
car_data = car_data.reindex(np.random.permutation(car_data.index))
# or equivalently,
# car_data = car_data.sample(frac=1)

In [6]:
print("Data set loaded. Num examples: ", len(car_data))

Data set loaded. Num examples:  205


In [7]:
car_data.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
182,2,122,volkswagen,diesel,std,two,sedan,fwd,front,97.30,...,97,idi,3.01,3.40,23.00,52,4800,37,46,7775
37,0,106,honda,gas,std,two,hatchback,fwd,front,96.50,...,110,1bbl,3.15,3.58,9.00,86,5800,27,33,7895
60,0,115,mazda,gas,std,four,sedan,fwd,front,98.80,...,122,2bbl,3.39,3.39,8.60,84,4800,26,32,8495
114,0,?,peugot,diesel,turbo,four,wagon,rwd,front,114.20,...,152,idi,3.70,3.52,21.00,95,4150,25,25,17075
71,-1,?,mercedes-benz,gas,std,four,sedan,rwd,front,115.60,...,234,mpfi,3.46,3.10,8.30,155,4750,16,18,34184


This is a really small dataset! Only 205 examples.

For simplicity in this codelab, we do not split the data further into training and validation. But you MUST do this on real datasets, or else you will overfit to your single dataset.

## Task 0: Use pandas to explore and prepare the data

- Use Pandas to inspect the data and manually curate a list of numeric_feature_names and categorical_feature_names.


Useful functions:
- `type()` called on any Python object describes the type of the object
- `dataframe[4:7]` pulls out rows 4, 5, 6 in a Pandas dataframe
- `dataframe[['mycol1', 'mycol2']]` pulls out the two requested columns into a new Pandas dataframe
- `dataframe['mycol1']` returns a Pandas series -- not a dataframe!
- `dataframe.describe()` prints out statistics for each dataframe column

In [8]:
car_data[4:7]

,symboling,normalized-losses,make,fuel-type,aspiration,num-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
71,-1,?,mercedes-benz,gas,std,four,sedan,rwd,front,115.60,...,234,mpfi,3.46,3.10,8.30,155,4750,16,18,34184
39,0,85,honda,gas,std,four,sedan,fwd,front,96.50,...,110,1bbl,3.15,3.58,9.00,86,5800,27,33,8845
175,-1,65,toyota,gas,std,four,hatchback,fwd,front,102.40,...,122,mpfi,3.31,3.54,8.70,92,4200,27,32,9988


In [9]:
car_data.describe()

,symboling,wheel-base,length,width,height,weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,10.14,25.22,30.75
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,3.97,6.54,6.89
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,7.00,13.00,16.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,8.60,19.00,25.00
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,9.00,24.00,30.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,9.40,30.00,34.00
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,23.00,49.00,54.00


In [10]:
LABEL = 'price'

numeric_feature_names = ['symboling', 'normalized-losses', 'wheel-base',
        'length', 'width', 'height', 'weight', 'engine-size', 'horsepower',
        'peak-rpm', 'city-mpg', 'highway-mpg', 'bore', 'stroke',
         'compression-ratio']

categorical_feature_names = list(set(feature_names) - set(numeric_feature_names) - set([LABEL]))

categorical_feature_names

assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [11]:
# The correct solution will pass these assert statements.
assert len(numeric_feature_names) == 15
assert len(categorical_feature_names) == 10

In [12]:
# Run to inspect numeric features.
car_data[numeric_feature_names]

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio
182,2,122,97.30,171.70,65.50,55.70,2261,97,52,4800,37,46,3.01,3.40,23.00
37,0,106,96.50,167.50,65.20,53.30,2236,110,86,5800,27,33,3.15,3.58,9.00
60,0,115,98.80,177.80,66.50,55.50,2410,122,84,4800,26,32,3.39,3.39,8.60
114,0,?,114.20,198.90,68.40,58.70,3485,152,95,4150,25,25,3.70,3.52,21.00
71,-1,?,115.60,202.60,71.70,56.50,3740,234,155,4750,16,18,3.46,3.10,8.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,3,?,95.90,173.20,66.30,50.20,2921,156,145,5000,19,24,3.59,3.86,7.00
29,3,145,95.90,173.20,66.30,50.20,2811,156,145,5000,19,24,3.60,3.90,7.00
199,-1,74,104.30,188.80,67.20,57.50,3157,130,162,5100,17,22,3.62,3.15,7.50
109,0,?,114.20,198.90,68.40,58.70,3230,120,97,5000,19,24,3.46,3.19,8.40


In [13]:
car_data[numeric_feature_names].describe()

,symboling,wheel-base,length,width,height,weight,engine-size,city-mpg,highway-mpg,compression-ratio
count,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00,205.00
mean,0.83,98.76,174.05,65.91,53.72,2555.57,126.91,25.22,30.75,10.14
std,1.25,6.02,12.34,2.15,2.44,520.68,41.64,6.54,6.89,3.97
min,-2.00,86.60,141.10,60.30,47.80,1488.00,61.00,13.00,16.00,7.00
25%,0.00,94.50,166.30,64.10,52.00,2145.00,97.00,19.00,25.00,8.60
50%,1.00,97.00,173.20,65.50,54.10,2414.00,120.00,24.00,30.00,9.00
75%,2.00,102.40,183.10,66.90,55.50,2935.00,141.00,30.00,34.00,9.40
max,3.00,120.90,208.10,72.30,59.80,4066.00,326.00,49.00,54.00,23.00


In [14]:
# Coerce the numeric features to numbers. This is necessary because the model crashes because not all the values are numeric.
for feature_name in numeric_feature_names + [LABEL]:
    car_data[feature_name] = pd.to_numeric(car_data[feature_name], errors ='coerce')

"""
errors: error는 총 3개의 옵션이 존재합니다.
- errors = 'ignore' -> 만약 숫자로 변경할 수 없는 데이터라면 숫자로 변경하지 않고 원본 데이터를 그대로 반환합니다.
- errors = 'coerce' -> 만약 숫자로 변경할 수 없는 데이터라면 기존 데이터를 지우고 NaN으로 설정하여 반환합니다.
- errors = 'raise' -> 만약 숫자로 변경할 수 없는 데이터라면 에러를 일으키며 코드를 중단합니다."""
    
# Fill missing values with 0.
# Is this an OK thing to do? You may want to come back and revisit this decision later.
car_data.fillna(0, inplace=True)

## Task 1: Make your best model with numeric features. No normalization allowed.

Modify the model provided below to achieve the lowest eval loss. You may want to change various hyperparameters:
- learning rate
- choice of optimizer
- hidden layer dimensions -- make sure your choice here makes sense given the number of training examples
- batch size
- num training steps
- (anything else you can think of changing)

Do not use the `normalizer_fn` arg on `numeric_column`.

In [15]:
car_data.shape

(205, 26)

In [16]:
X = car_data[numeric_feature_names].copy()
y = car_data['price'].copy()

In [17]:
X.columns

Index(['symboling', 'normalized-losses', 'wheel-base', 'length', 'width',
       'height', 'weight', 'engine-size', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'bore', 'stroke', 'compression-ratio'],
      dtype='object')

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2 )

In [19]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=(15,), activation='relu'))
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [20]:
batch_size=16
epochs=500
model.fit(X_train, y_train, epochs=epochs, batch_size= batch_size)

Epoch 1/500
11/11 [==============================] - 1s 2ms/step - loss: 223663056.0000 - mean_squared_error: 223663056.0000
Epoch 2/500
11/11 [==============================] - 0s 2ms/step - loss: 223660288.0000 - mean_squared_error: 223660288.0000
Epoch 3/500
11/11 [==============================] - 0s 2ms/step - loss: 223657440.0000 - mean_squared_error: 223657440.0000
Epoch 4/500
11/11 [==============================] - 0s 3ms/step - loss: 223654720.0000 - mean_squared_error: 223654720.0000
Epoch 5/500
11/11 [==============================] - 0s 3ms/step - loss: 223651920.0000 - mean_squared_error: 223651920.0000
Epoch 6/500
11/11 [==============================] - 0s 3ms/step - loss: 223649216.0000 - mean_squared_error: 223649216.0000
Epoch 7/500
11/11 [==============================] - 0s 2ms/step - loss: 223646400.0000 - mean_squared_error: 223646400.0000
Epoch 8/500
11/11 [==============================] - 0s 2ms/step - loss: 223643600.0000 - mean_squared_error: 223643600.0000


11/11 [==============================] - 0s 2ms/step - loss: 223483360.0000 - mean_squared_error: 223483360.0000
Epoch 67/500
11/11 [==============================] - 0s 2ms/step - loss: 223480528.0000 - mean_squared_error: 223480528.0000
Epoch 68/500
11/11 [==============================] - 0s 2ms/step - loss: 223477840.0000 - mean_squared_error: 223477840.0000
Epoch 69/500
11/11 [==============================] - 0s 2ms/step - loss: 223475040.0000 - mean_squared_error: 223475040.0000
Epoch 70/500
11/11 [==============================] - 0s 2ms/step - loss: 223472320.0000 - mean_squared_error: 223472320.0000
Epoch 71/500
11/11 [==============================] - 0s 2ms/step - loss: 223469648.0000 - mean_squared_error: 223469648.0000
Epoch 72/500
11/11 [==============================] - 0s 3ms/step - loss: 223466864.0000 - mean_squared_error: 223466864.0000
Epoch 73/500
11/11 [==============================] - 0s 3ms/step - loss: 223464176.0000 - mean_squared_error: 223464176.0000
Epoch

11/11 [==============================] - 0s 2ms/step - loss: 223306800.0000 - mean_squared_error: 223306800.0000
Epoch 131/500
11/11 [==============================] - 0s 2ms/step - loss: 223303904.0000 - mean_squared_error: 223303904.0000
Epoch 132/500
11/11 [==============================] - 0s 2ms/step - loss: 223301152.0000 - mean_squared_error: 223301152.0000
Epoch 133/500
11/11 [==============================] - 0s 2ms/step - loss: 223298384.0000 - mean_squared_error: 223298384.0000
Epoch 134/500
11/11 [==============================] - 0s 2ms/step - loss: 223295568.0000 - mean_squared_error: 223295568.0000
Epoch 135/500
11/11 [==============================] - 0s 2ms/step - loss: 223292816.0000 - mean_squared_error: 223292816.0000
Epoch 136/500
11/11 [==============================] - 0s 3ms/step - loss: 223290064.0000 - mean_squared_error: 223290064.0000
Epoch 137/500
11/11 [==============================] - 0s 4ms/step - loss: 223287264.0000 - mean_squared_error: 223287264.000

11/11 [==============================] - 0s 4ms/step - loss: 223133104.0000 - mean_squared_error: 223133104.0000
Epoch 194/500
11/11 [==============================] - 0s 3ms/step - loss: 223130272.0000 - mean_squared_error: 223130272.0000
Epoch 195/500
11/11 [==============================] - 0s 3ms/step - loss: 223127456.0000 - mean_squared_error: 223127456.0000
Epoch 196/500
11/11 [==============================] - 0s 4ms/step - loss: 223124704.0000 - mean_squared_error: 223124704.0000
Epoch 197/500
11/11 [==============================] - 0s 3ms/step - loss: 223121984.0000 - mean_squared_error: 223121984.0000
Epoch 198/500
11/11 [==============================] - 0s 2ms/step - loss: 223119232.0000 - mean_squared_error: 223119232.0000
Epoch 199/500
11/11 [==============================] - 0s 2ms/step - loss: 223116464.0000 - mean_squared_error: 223116464.0000
Epoch 200/500
11/11 [==============================] - 0s 3ms/step - loss: 223113808.0000 - mean_squared_error: 223113808.000

11/11 [==============================] - 0s 2ms/step - loss: 222960064.0000 - mean_squared_error: 222960064.0000
Epoch 257/500
11/11 [==============================] - 0s 2ms/step - loss: 222957312.0000 - mean_squared_error: 222957312.0000
Epoch 258/500
11/11 [==============================] - 0s 3ms/step - loss: 222954624.0000 - mean_squared_error: 222954624.0000
Epoch 259/500
11/11 [==============================] - 0s 2ms/step - loss: 222952000.0000 - mean_squared_error: 222952000.0000
Epoch 260/500
11/11 [==============================] - 0s 2ms/step - loss: 222949200.0000 - mean_squared_error: 222949200.0000
Epoch 261/500
11/11 [==============================] - 0s 2ms/step - loss: 222946528.0000 - mean_squared_error: 222946528.0000
Epoch 262/500
11/11 [==============================] - 0s 2ms/step - loss: 222943728.0000 - mean_squared_error: 222943728.0000
Epoch 263/500
11/11 [==============================] - 0s 3ms/step - loss: 222941008.0000 - mean_squared_error: 222941008.000

11/11 [==============================] - 0s 2ms/step - loss: 222784336.0000 - mean_squared_error: 222784336.0000
Epoch 321/500
11/11 [==============================] - 0s 2ms/step - loss: 222781584.0000 - mean_squared_error: 222781584.0000
Epoch 322/500
11/11 [==============================] - 0s 2ms/step - loss: 222778864.0000 - mean_squared_error: 222778864.0000
Epoch 323/500
11/11 [==============================] - 0s 2ms/step - loss: 222776000.0000 - mean_squared_error: 222776000.0000
Epoch 324/500
11/11 [==============================] - 0s 2ms/step - loss: 222773280.0000 - mean_squared_error: 222773280.0000
Epoch 325/500
11/11 [==============================] - 0s 2ms/step - loss: 222770448.0000 - mean_squared_error: 222770448.0000
Epoch 326/500
11/11 [==============================] - 0s 2ms/step - loss: 222767680.0000 - mean_squared_error: 222767680.0000
Epoch 327/500
11/11 [==============================] - 0s 2ms/step - loss: 222764976.0000 - mean_squared_error: 222764976.000

11/11 [==============================] - 0s 2ms/step - loss: 222608160.0000 - mean_squared_error: 222608160.0000
Epoch 385/500
11/11 [==============================] - 0s 2ms/step - loss: 222605456.0000 - mean_squared_error: 222605456.0000
Epoch 386/500
11/11 [==============================] - 0s 2ms/step - loss: 222602736.0000 - mean_squared_error: 222602736.0000
Epoch 387/500
11/11 [==============================] - 0s 3ms/step - loss: 222600064.0000 - mean_squared_error: 222600064.0000
Epoch 388/500
11/11 [==============================] - 0s 2ms/step - loss: 222597264.0000 - mean_squared_error: 222597264.0000
Epoch 389/500
11/11 [==============================] - 0s 2ms/step - loss: 222594480.0000 - mean_squared_error: 222594480.0000
Epoch 390/500
11/11 [==============================] - 0s 2ms/step - loss: 222591680.0000 - mean_squared_error: 222591680.0000
Epoch 391/500
11/11 [==============================] - 0s 2ms/step - loss: 222589072.0000 - mean_squared_error: 222589072.000

11/11 [==============================] - 0s 2ms/step - loss: 222432000.0000 - mean_squared_error: 222432000.0000
Epoch 449/500
11/11 [==============================] - 0s 2ms/step - loss: 222429264.0000 - mean_squared_error: 222429264.0000
Epoch 450/500
11/11 [==============================] - 0s 3ms/step - loss: 222426512.0000 - mean_squared_error: 222426512.0000
Epoch 451/500
11/11 [==============================] - 0s 2ms/step - loss: 222423712.0000 - mean_squared_error: 222423712.0000
Epoch 452/500
11/11 [==============================] - 0s 2ms/step - loss: 222421072.0000 - mean_squared_error: 222421072.0000
Epoch 453/500
11/11 [==============================] - 0s 2ms/step - loss: 222418368.0000 - mean_squared_error: 222418368.0000
Epoch 454/500
11/11 [==============================] - 0s 2ms/step - loss: 222415600.0000 - mean_squared_error: 222415600.0000
Epoch 455/500
11/11 [==============================] - ETA: 0s - loss: 211770016.0000 - mean_squared_error: 211770016.00 - 0s

In [21]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 267043312.0000 - mean_squared_error: 267043312.0000


[267043312.0, 267043312.0]

### normalization

In [22]:
car_data_norm = pd.DataFrame()
for feature_name in numeric_feature_names + [LABEL]:
    car_data_norm[feature_name] = (car_data[feature_name]-car_data[feature_name].mean())/car_data[feature_name].std()

In [23]:
car_data_norm.head()

,symboling,normalized-losses,wheel-base,length,width,height,weight,engine-size,horsepower,peak-rpm,city-mpg,highway-mpg,bore,stroke,compression-ratio,price
182,0.94,0.42,-0.24,-0.19,-0.19,0.81,-0.57,-0.72,-1.25,-0.40,1.80,2.21,-0.48,0.38,3.24,-0.64
37,-0.67,0.14,-0.37,-0.53,-0.33,-0.17,-0.61,-0.41,-0.42,1.04,0.27,0.33,-0.21,0.71,-0.29,-0.63
60,-0.67,0.30,0.01,0.30,0.28,0.73,-0.28,-0.12,-0.47,-0.40,0.12,0.18,0.23,0.36,-0.39,-0.55
114,-0.67,-1.67,2.56,2.01,1.16,2.04,1.79,0.60,-0.20,-1.33,-0.03,-0.84,0.81,0.60,2.73,0.51
71,-1.47,-1.67,2.80,2.31,2.70,1.14,2.27,2.57,1.27,-0.47,-1.41,-1.85,0.36,-0.17,-0.46,2.63


In [24]:
X_norm = car_data_norm[numeric_feature_names].copy()
y_norm = car_data_norm['price'].copy()

In [25]:
Xn_train, Xn_test, yn_train, yn_test = train_test_split(X_norm, y_norm, test_size=0.2)

In [26]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=(15,), activation='relu'))
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [27]:
batch_size=16
epochs=500
model.fit(Xn_train, yn_train, epochs=epochs, batch_size= batch_size)

Epoch 1/500
11/11 [==============================] - 1s 3ms/step - loss: 2.2123 - mean_squared_error: 2.2123
Epoch 2/500
11/11 [==============================] - 0s 3ms/step - loss: 1.1374 - mean_squared_error: 1.1374
Epoch 3/500
11/11 [==============================] - 0s 3ms/step - loss: 0.9210 - mean_squared_error: 0.9210
Epoch 4/500
11/11 [==============================] - 0s 5ms/step - loss: 0.7680 - mean_squared_error: 0.7680
Epoch 5/500
11/11 [==============================] - 0s 3ms/step - loss: 0.5736 - mean_squared_error: 0.5736
Epoch 6/500
11/11 [==============================] - 0s 3ms/step - loss: 0.4435 - mean_squared_error: 0.4435
Epoch 7/500
11/11 [==============================] - 0s 3ms/step - loss: 0.3738 - mean_squared_error: 0.3738
Epoch 8/500
11/11 [==============================] - ETA: 0s - loss: 0.4957 - mean_squared_error: 0.49 - 0s 3ms/step - loss: 0.3208 - mean_squared_error: 0.3208
Epoch 9/500
11/11 [==============================] - 0s 4ms/step - loss: 0.2

11/11 [==============================] - 0s 2ms/step - loss: 0.1014 - mean_squared_error: 0.1014
Epoch 75/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0975 - mean_squared_error: 0.0975
Epoch 76/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1033 - mean_squared_error: 0.1033
Epoch 77/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1035 - mean_squared_error: 0.1035
Epoch 78/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1036 - mean_squared_error: 0.1036
Epoch 79/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1029 - mean_squared_error: 0.1029
Epoch 80/500
11/11 [==============================] - 0s 2ms/step - loss: 0.1007 - mean_squared_error: 0.1007
Epoch 81/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0976 - mean_squared_error: 0.0976
Epoch 82/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0972 - mean_squared_error: 0.0972
Epoch 83/500
11/11 [===

11/11 [==============================] - 0s 3ms/step - loss: 0.0809 - mean_squared_error: 0.0809
Epoch 148/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0849 - mean_squared_error: 0.0849
Epoch 149/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0829 - mean_squared_error: 0.0829
Epoch 150/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0841 - mean_squared_error: 0.0841
Epoch 151/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0804 - mean_squared_error: 0.0804
Epoch 152/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0805 - mean_squared_error: 0.0805
Epoch 153/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0783 - mean_squared_error: 0.0783
Epoch 154/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0784 - mean_squared_error: 0.0784
Epoch 155/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0804 - mean_squared_error: 0.0804
Epoch 156/500
1

Epoch 220/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0742 - mean_squared_error: 0.0742
Epoch 221/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0739 - mean_squared_error: 0.0739
Epoch 222/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0756 - mean_squared_error: 0.0756
Epoch 223/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0737 - mean_squared_error: 0.0737
Epoch 224/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0748 - mean_squared_error: 0.0748
Epoch 225/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0786 - mean_squared_error: 0.0786
Epoch 226/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0756 - mean_squared_error: 0.0756
Epoch 227/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0749 - mean_squared_error: 0.0749
Epoch 228/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0752 - mean_squared_error: 0.0752
E

11/11 [==============================] - 0s 7ms/step - loss: 0.0827 - mean_squared_error: 0.0827
Epoch 294/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0901 - mean_squared_error: 0.0901
Epoch 295/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0759 - mean_squared_error: 0.0759
Epoch 296/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0714 - mean_squared_error: 0.0714
Epoch 297/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0673 - mean_squared_error: 0.0673
Epoch 298/500
11/11 [==============================] - ETA: 0s - loss: 0.0739 - mean_squared_error: 0.07 - 0s 3ms/step - loss: 0.0663 - mean_squared_error: 0.0663
Epoch 299/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0657 - mean_squared_error: 0.0657
Epoch 300/500
11/11 [==============================] - ETA: 0s - loss: 0.0267 - mean_squared_error: 0.02 - 0s 3ms/step - loss: 0.0675 - mean_squared_error: 0.0675
Epoch 301/500
11/11 [=

11/11 [==============================] - 0s 3ms/step - loss: 0.0645 - mean_squared_error: 0.0645
Epoch 366/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0636 - mean_squared_error: 0.0636
Epoch 367/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0638 - mean_squared_error: 0.0638
Epoch 368/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0623 - mean_squared_error: 0.0623
Epoch 369/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0644 - mean_squared_error: 0.0644
Epoch 370/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0649 - mean_squared_error: 0.0649
Epoch 371/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0615 - mean_squared_error: 0.0615
Epoch 372/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0611 - mean_squared_error: 0.0611
Epoch 373/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0618 - mean_squared_error: 0.0618
Epoch 374/500
1

11/11 [==============================] - 0s 4ms/step - loss: 0.0622 - mean_squared_error: 0.0622
Epoch 438/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0603 - mean_squared_error: 0.0603
Epoch 439/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0592 - mean_squared_error: 0.0592
Epoch 440/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0699 - mean_squared_error: 0.0699
Epoch 441/500
11/11 [==============================] - 0s 10ms/step - loss: 0.0614 - mean_squared_error: 0.0614
Epoch 442/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0574 - mean_squared_error: 0.0574
Epoch 443/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0585 - mean_squared_error: 0.0585
Epoch 444/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0597 - mean_squared_error: 0.0597
Epoch 445/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0577 - mean_squared_error: 0.0577
Epoch 446/500


In [28]:
model.evaluate(Xn_test, yn_test)

2/2 [==============================] - 1s 15ms/step - loss: 0.3737 - mean_squared_error: 0.3737


[0.3736814260482788, 0.3736814260482788]

In [29]:
car_data_minmax = pd.DataFrame()
for feature_name in numeric_feature_names + [LABEL]:
    car_data_minmax[feature_name] = (car_data[feature_name]-car_data[feature_name].min())/(car_data[feature_name].max()-car_data[feature_name].min())

In [30]:
Xm_norm = car_data_minmax[numeric_feature_names].copy()
ym_norm = car_data_minmax['price'].copy()

In [31]:
Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm_norm, ym_norm, test_size=0.2)

In [32]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=(15,), activation='relu'))
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

In [33]:
batch_size=16
epochs=500
model.fit(Xm_train, ym_train, epochs=epochs, batch_size= batch_size)

Epoch 1/500
11/11 [==============================] - 3s 5ms/step - loss: 0.2587 - mean_squared_error: 0.2587
Epoch 2/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0595 - mean_squared_error: 0.0595
Epoch 3/500
11/11 [==============================] - 0s 13ms/step - loss: 0.0402 - mean_squared_error: 0.0402
Epoch 4/500
11/11 [==============================] - 0s 15ms/step - loss: 0.0344 - mean_squared_error: 0.0344
Epoch 5/500
11/11 [==============================] - 0s 12ms/step - loss: 0.0338 - mean_squared_error: 0.0338
Epoch 6/500
11/11 [==============================] - 0s 9ms/step - loss: 0.0336 - mean_squared_error: 0.0336
Epoch 7/500
11/11 [==============================] - 0s 12ms/step - loss: 0.0336 - mean_squared_error: 0.0336
Epoch 8/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0335 - mean_squared_error: 0.0335
Epoch 9/500
11/11 [==============================] - 0s 12ms/step - loss: 0.0336 - mean_squared_error: 0.0336
Epoch 10/500
1

Epoch 75/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0073 - mean_squared_error: 0.0073
Epoch 76/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0079 - mean_squared_error: 0.0079
Epoch 77/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0077 - mean_squared_error: 0.0077
Epoch 78/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 79/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0071 - mean_squared_error: 0.0071
Epoch 80/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 81/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0072 - mean_squared_error: 0.0072
Epoch 82/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0069 - mean_squared_error: 0.0069
Epoch 83/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0067 - mean_squared_error: 0.0067
Epoch 84/5

Epoch 149/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 150/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0052 - mean_squared_error: 0.0052
Epoch 151/500
11/11 [==============================] - 0s 23ms/step - loss: 0.0055 - mean_squared_error: 0.0055
Epoch 152/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 153/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0053 - mean_squared_error: 0.0053
Epoch 154/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 155/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0056 - mean_squared_error: 0.0056
Epoch 156/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 157/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049


Epoch 222/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 223/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 224/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0048 - mean_squared_error: 0.0048
Epoch 225/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 226/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 227/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 228/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 229/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 230/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0045 - mean_squared_error: 0.0045
E

Epoch 295/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 296/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0051 - mean_squared_error: 0.0051
Epoch 297/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 298/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 299/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 300/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 301/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 302/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 303/500
11/11 [==============================] - 0s 6ms/step - loss: 0.0044 - mean_squared_error: 0.0044
E

11/11 [==============================] - 0s 3ms/step - loss: 0.0049 - mean_squared_error: 0.0049
Epoch 368/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 369/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 370/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 371/500
11/11 [==============================] - 0s 7ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 372/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 373/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 374/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 375/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 376/500
1

11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 441/500
11/11 [==============================] - 0s 2ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 442/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0045 - mean_squared_error: 0.0045
Epoch 443/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0047 - mean_squared_error: 0.0047
Epoch 444/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 445/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046
Epoch 446/500
11/11 [==============================] - 0s 3ms/step - loss: 0.0046 - mean_squared_error: 0.0046    
Epoch 447/500
11/11 [==============================] - 0s 5ms/step - loss: 0.0043 - mean_squared_error: 0.0043
Epoch 448/500
11/11 [==============================] - 0s 4ms/step - loss: 0.0044 - mean_squared_error: 0.0044
Epoch 449/5

In [34]:
model.evaluate(Xm_test, ym_test)

2/2 [==============================] - 1s 6ms/step - loss: 0.0252 - mean_squared_error: 0.0252


[0.025208203122019768, 0.025208203122019768]

In [41]:
feature_columns = []
# for feature_name in numeric_feature_names + [LABEL]:
for feature_name in numeric_feature_names:
    feature_columns.append(tf.feature_column.numeric_column(feature_name))
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [50]:
model = tf.keras.models.Sequential()
model.add(feature_layer)
model.add(tf.keras.layers.Dense(units=3, input_shape=(1,), activation='relu'))
# model.add(tf.keras.layers.Dense(units=1, activation='relu')) 
#                         feature_layer일 경우 input_dim은 무조건 1?
model.add(tf.keras.layers.Dense(units=1))
model.compile(tf.keras.optimizers.Adam(learning_rate=0.01),
             loss='mean_squared_error',
             metrics=[tf.keras.metrics.MeanSquaredError()])

### convert dataframe to dict
when using feature_layer, <span style='color:red'> use dict </span> for input data in model.fit

In [47]:
Xm_train_dict = {name:np.array(value) for name, value in Xm_train.items()}
ym_train_dict = {name:np.array(value) for name, value in ym_train.items()}

In [48]:
len(Xm_train_dict['symboling'])

164

In [51]:
batch_size=16
epochs=500
model.fit(Xm_train_dict, ym_train_dict, epochs=epochs, batch_size= batch_size)

IndexError: list index out of range

In [ ]:
Xm_train.head()